# GGA 核坐标梯度性质

在这一节中，我们将会进一步拓展核坐标梯度性质的学习，将 GGA 引入 SCF，获得 GGA 的一阶核梯度与二阶核梯度的性质．这里所指的 GGA 是以 GGA 为基础的泛函，包括杂化的与非杂化的．双杂化的 GGA 将会在以后讨论．

一般来说，近 30 年的中小规模分子量化计算的主流始终是 DFT 近似．这源于 DFT 近似的计算量与 HF 相当，但结果通常比 HF 优异许多，可以媲美最低阶的 Post-HF 方法 MP2 的结果．

DFT 近似各式各样；一般来说，化学领域的最主流是 B3LYP，而物理领域的最主流是 PBE；这两者都是以 GGA 为基础的泛函．在这一节中，我们会以 B3LYP 为例，了解运用 PySCF 的底层函数来解决 B3LYP 的一阶梯度与二阶梯度．这里指的 PySCF 底层函数是指求取 DFT 积分格点的函数．

关于 B3LYP 的能量的计算与调用细节，可以前往 [XYG3 能量](xyg3_energy.ipynb#输出-2：交换相关能格点求取) 文档的其中一小节．这一节的记号会稍有不同，但应当容易理解．许多基础的公式与程序细节这一节不再叙述．

由于 DFT 格点积分的计算中，会使用内存消耗上巨大的格点；在小体系下，格点的数量会远大于基组与原子数量，因此处理计算时，不谨慎的算法与数据存储很可能导致巨大的计算消耗，或者巨大的内存占用．事实上，在我自己编写与调试这份文档时，已经出现过内存占用过大而宕机的情况．因此，不少代码块将会显示执行时间，以对代码效率有直观的认识．不过，`%%time` 只能给出定性效率；定量的效率仍然需要通过 `%%timeit` 给出．

In [ ]:
from pyscf import gto, scf, dft, grad, data, lib, hessian
import numpy as np

import pyscf.grad.rks
import pyscf.hessian.rks
np.set_printoptions(5, linewidth=150, suppress=True)

## 准备工作

### 顶层函数计算 RKS 梯度

In [ ]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.0  0.0  0.0
H  0.0  1.0  1.5
"""
mol.basis = "6-31G"
mol.build()

In [ ]:
%%time
scf_eng = dft.RKS(mol)
scf_eng.xc = "b3lypg"  # compare that to gaussian
scf_eng.grids.atom_grid = (99, 590)
scf_eng.grids.build()
scf_eng.conv_tol = 1e-13
energy_RKS = scf_eng.kernel()

In [ ]:
%%time
scf_grad = grad.rks.Gradients(scf_eng)
grad_RKS = scf_grad.kernel()

In [ ]:
%%time
scf_hess = hessian.rks.Hessian(scf_eng)
hess_RKS = scf_hess.kernel()

### 与 Gaussian 结果进行比对

In [ ]:
def val_from_fchk(key, file_path):
    flag_read = False; expect_size = -1; vec = []
    with open(file_path, "r") as file:
        for l in file:
            if (l[:len(key)] == key):
                try: expect_size = int(l[len(key):].split()[2]); flag_read = True; continue
                except IndexError: return float(l[len(key):].split()[1])
            if (flag_read):
                try: vec += [ float(i) for i in l.split() ]
                except ValueError: break
    if len(vec) != expect_size: raise ValueError("Number of expected size is not consistent with read-in size!")
    return np.array(vec)

In [ ]:
energy_Gaussian = val_from_fchk("SCF Energy", "include/B3LYP-hess.fch")
grad_Gaussian = val_from_fchk("Cartesian Gradient", "include/B3LYP-hess.fch")
hess_Gaussian = val_from_fchk("Cartesian Force Constants", "include/B3LYP-hess.fch")

由于 PySCF 的 DFT 积分与 Gaussian 仍然似乎略有不同，因此尽管数据大体能对上，但随着导数的阶越大，偏差也越大；二阶梯度在这两个程序的匹配精度只有 $10^{-4}$ 级别．

In [ ]:
print(np.allclose(energy_RKS, energy_Gaussian))
print(np.allclose(grad_RKS.reshape(-1), grad_Gaussian, atol=1e-5))
d_hess = mol.natm * 3
print(np.allclose(hess_RKS.swapaxes(1, 2).reshape(d_hess, d_hess)[np.tril_indices(d_hess)], hess_Gaussian, atol=1e-4))

### RKS 重要中间量

In [ ]:
c_x = scf_eng._numint.hybrid_coeff(scf_eng.xc)

nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]
mo_occ = scf_eng.mo_occ

D = scf_eng.make_rdm1()
De = np.einsum("ui, i, vi -> uv", C, e * mo_occ, C)

In [ ]:
natm = mol.natm

# grad-contrib
int1e_ipovlp = mol.intor('int1e_ipovlp')
int1e_ipkin = mol.intor("int1e_ipkin")
int1e_ipnuc = mol.intor("int1e_ipnuc")
int2e_ip1 = mol.intor("int2e_ip1")

# hess-contrib
int1e_ipipkin = mol.intor("int1e_ipipkin")
int1e_ipkinip = mol.intor("int1e_ipkinip")
int1e_ipipnuc = mol.intor("int1e_ipipnuc")
int1e_ipnucip = mol.intor("int1e_ipnucip")
int2e_ipip1 = mol.intor("int2e_ipip1")
int2e_ipvip1 = mol.intor("int2e_ipvip1")
int2e_ip1ip2 = mol.intor("int2e_ip1ip2")
int1e_ipipovlp = mol.intor("int1e_ipipovlp")
int1e_ipovlpip = mol.intor("int1e_ipovlpip")

In [ ]:
def mol_slice(atm_id):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    return slice(p0, p1)

### GGA 梯度所需要的函数格点

这篇笔记会着重叙述 GGA 的二阶梯度．尽管 GGA 的二阶梯度的计算公式未必很多，但需要定义大量的格点张量，这里简单地先作一个回顾与总结．

先定义一些记号．由于后面将会大量碰到对轨道 $\phi_{\mu}$、泛函核 $f_{g}^\mathrm{xc}$ 的偏导数，记号会变得非常麻烦．为了简记记号，让公式看起来更简洁，但又不影响公式的意义，以及公式到程序的对应，我们使用以下记号及其衍生记号：

* $\phi_{tsrg \mu}$ = $\partial_t \partial_s \partial_r \phi_{g \mu}$；以后，$t, s, r, w$ 放在下标时表示三维坐标分量，$g$ 代表格点；一般来说，$t, s$ 是与从原子坐标分量的导数产生的电子坐标分量；而 $r, w$ 则是 $\partial_{\nabla \rho} f_g^\mathrm{xc} = 2 f_{\gamma g}^\mathrm{xc} \nabla \rho$ 中 $\nabla$ 所指代的电子坐标分量；$w$ 在其它语境下可能表示格点权重 $w_g$；

* $\phi_{t \mu} = \partial_t \phi_{\mu} (\boldsymbol{r})$；一般地不含格点的记号表示关于 $\boldsymbol{r}$ 的函数；除非显式地写出电子坐标，譬如 $\phi_{t \mu} (\boldsymbol{w})$；

* $f_{g}^{\rho \gamma}$ = $\partial_\rho \partial_\gamma f_{g}^\mathrm{xc}$；去掉了 $\mathrm{xc}$ 记号；

* $\rho_{t} = \partial_t \rho = D_{\mu \nu} (\phi_{t \mu} \phi_{\nu} + \phi_{\mu} \phi_{t \nu}) = 2 D_{\mu \nu} \phi_{t \mu} \phi_{\nu}$；

* $\rho_{A t} = - \partial_{A_t} \rho = 2 D_{\mu_A \nu} \phi_{t \mu_A} \phi_{\nu}$，它表示密度 $\rho$ 在电子坐标分量 $t$ 的导数；但被求导的原子轨道必须属于原子 $A$．由于负号通常不是很友好，并且这一项会非常经常地出现在公式中，因此为了便利定义该记号；

* $\rho_{Atr} = \partial_{r} \rho_{At} = 2 D_{\mu_A \nu} (\phi_{tr \mu_A} \phi_{\nu} + \phi_{t \mu_A} \phi_{r \nu})$；其它高次密度偏导数同理．

<div class="alert alert-warning">

**注意**

关于偏导符号，这里需要作说明．在求梯度时，我们经常地需要区分 U 矩阵与非 U 矩阵 (Skeleton/Core) 的贡献．在这份文档中，分别用简化偏导记号 $\partial_{t}$ 与 $\partial_t^\mathrm{U}$ 区分两种贡献．举例而言，对于 $\rho$ 而言，其对 $t$ 的完整偏导数为

\begin{align}
(\partial_t + \partial_t^\mathrm{U}) \rho &= D_{\mu \nu} \partial_t (\phi_\mu \phi_\nu) + \phi_\mu \phi_\nu \partial_t^\mathrm{U} (2 C_{\mu i} C_{\nu i}) \\
&= D_{\mu \nu} (\phi_{t \mu} \phi_\nu + \phi_\mu \phi_{t \nu}) + 2 U_{pi} (C_{\mu p} C_{\nu i} + C_{\mu i} C_{\nu p}) \phi_\mu \phi_\nu \\
&= 2 D_{\mu \nu} \phi_{t \mu} \phi_\nu + 4 U_{pi} C_{\mu p} C_{\nu i} \phi_\mu \phi_\nu
\end{align}

</div>

首先，我们拿出格点原子轨道函数值、蒙版、权重与坐标．蒙版的作用是在 PySCF 中加快计算速度，我们在这里暂还不需要这个向量；格点坐标在梯度公式中也用不上．需要注意，下面的代码只用于演示用途；如 [以前文档](xyg3_energy.ipynb#DFT-计算使用的格点) 所述，在体系变大时， PySCF 会使用迭代器，多次生成轨道格点．目前体系下，一次性生成所有 (包含三阶导的) 轨道格点需要占用约 1GB 的内存．我们在下述函数中，提供大约 2GB 内存来生成所有轨道格点：

In [ ]:
%%time
grid_ao, grid_mask, grid_weight, grid_coord = next(scf_eng._numint.block_loop(mol, scf_eng.grids, nao, 3, 2000))
ngrid = grid_ao.shape[1]

上述的轨道格点的顺序可以参考 [PySCF 文档](https://sunqm.github.io/pyscf/dft.html#pyscf.dft.numint.eval_ao)．但为了公式与代码对应上的便利，我们会在代码中重新定义一些张量．

* `grid_ao_0` : $\phi_{gu}$

* `grid_ao_1` : $\phi_{tgu}$

* `grid_ao_2` : $\phi_{tsgu}$

* `grid_ao_2T` : $\phi_{Tgu}$

* `grid_ao_3T` : $\phi_{Trgu}$

上面的公式中，$T$ 指代两个坐标分量 $ts$ 的导数；即 $\phi_{Trgu}$ 与 $\phi_{tsrgu}$ 是等价的．之所以可以这么定义，是因为在 GGA 二阶梯度中，所有轨道三阶导数只会与二阶导数进行张量积；张量积中总会对其中两个维度 $ts$ 求和．

In [ ]:
%%time
grid_ao_0  = grid_ao[0]
grid_ao_1  = grid_ao[1:4]
grid_ao_2T = grid_ao[4:10]

需要注意的是，`grid_ao_2` 与 `grid_ao_3T` 不能直接从轨道函数格点 `grid_ao` 中用 slice 直接截取，因此需要进行重定义的工作．对如此大小的张量进行重定义的工作实际上极为消耗内存与 I/O 时间，因此在编写实际程序时，下述的代码需要尽量避免．

In [ ]:
%%time
XX, XY, XZ, YY, YZ, ZZ = range(4, 10)
XXX, XXY, XXZ, XYY, XYZ, XZZ, YYY, YYZ, YZZ, ZZZ = range(10, 20)

grid_ao_2 = np.array([
    [grid_ao[XX], grid_ao[XY], grid_ao[XZ]],
    [grid_ao[XY], grid_ao[YY], grid_ao[YZ]],
    [grid_ao[XZ], grid_ao[YZ], grid_ao[ZZ]],
])
grid_ao_3T = np.array([
    [grid_ao[XXX], grid_ao[XXY], grid_ao[XXZ], grid_ao[XYY], grid_ao[XYZ], grid_ao[XZZ]],
    [grid_ao[XXY], grid_ao[XYY], grid_ao[XYZ], grid_ao[YYY], grid_ao[YYZ], grid_ao[YZZ]],
    [grid_ao[XXZ], grid_ao[XYZ], grid_ao[XZZ], grid_ao[YYZ], grid_ao[YZZ], grid_ao[ZZZ]],
])

可能我们会问，为何不用 list 重定义轨道二、三阶导数张量呢？似乎耗时更少：(事实上相当于在 C++ 中定义了一个新的引用数组，几乎没有 I/O)

In [ ]:
%%time
XX, XY, XZ, YY, YZ, ZZ = range(4, 10)
XXX, XXY, XXZ, XYY, XYZ, XZZ, YYY, YYZ, YZZ, ZZZ = range(10, 20)

grid_ao_2_list = [
    [grid_ao[XX], grid_ao[XY], grid_ao[XZ]],
    [grid_ao[XY], grid_ao[YY], grid_ao[YZ]],
    [grid_ao[XZ], grid_ao[YZ], grid_ao[ZZ]],
]
grid_ao_3T_list = [
    [grid_ao[XXX], grid_ao[XXY], grid_ao[XXZ], grid_ao[XYY], grid_ao[XYZ], grid_ao[XZZ]],
    [grid_ao[XXY], grid_ao[XYY], grid_ao[XYZ], grid_ao[YYY], grid_ao[YYZ], grid_ao[YZZ]],
    [grid_ao[XXZ], grid_ao[XYZ], grid_ao[XZZ], grid_ao[YYZ], grid_ao[YZZ], grid_ao[ZZZ]],
]

但通过下述三个代码块，我们可以知道，在执行 einsum 函数时，使用 np.array 初始化的 `grid_ao_2` 执行速度比使用 list 初始化的 `grid_ao_2_list` 快一些；且提速量大约是两次 `grid_ao_2_list` 的 np.array 初始化．

In [ ]:
%%timeit -r 3 -n 3
np.einsum("tsgu, tsgv -> uv", grid_ao_2, grid_ao_2)

In [ ]:
%%timeit -r 3 -n 3
np.einsum("tsgu, tsgv -> uv", grid_ao_2_list, grid_ao_2_list)

In [ ]:
%%timeit -r 3 -n 3
np.array(grid_ao_2_list); np.array(grid_ao_2_list)

因此，为了避免以后多次初始化大块张量，早期多花一些时间在一次性的 np.array 初始化向量会更好一些．

随后，我们定义密度与密度梯度的格点；这些量不仅在后续计算中经常用到，还用于生成泛函核 $f^\mathrm{xc} (\rho, \gamma)$ 的导数：

* `grid_rho_0` : $\rho_g = D_{\mu \nu} \phi_\mu \phi_\nu$

* `grid_rho_1` : $\rho_{tg} = 2 D_{\mu \nu} \phi_{t \mu} \phi_{\nu} = D_{\mu \nu} (\phi_{\mu} \phi_{t \nu})$

注意 $\rho_{tg}$ 已经对 $\mu, \nu$ 求和，并且 $D_{\mu \nu}$ 是对称矩阵，因此交换 $\mu \nu$ 角标是允许的，从而产生 2 倍．

In [ ]:
%%time
grid_rho = np.einsum("uv, tgu, gv -> tg", D, grid_ao[0:4], grid_ao_0, optimize=True)
grid_rho[1:] *= 2
grid_rho_0 = grid_rho[0]
grid_rho_1 = grid_rho[1:4]

* `grid_fr`  : $w_g f_g^{\rho}$
* `grid_fg`  : $w_g f_g^{\gamma}$
* `grid_frr` : $w_g f_g^{\rho \rho}$
* `grid_frg` : $w_g f_g^{\rho \gamma}$
* `grid_fgg` : $w_g f_g^{\gamma \gamma}$

In [ ]:
%%time
grid_vxc, grid_fxc = scf_eng._numint.eval_xc(scf_eng.xc, grid_rho, deriv=2)[1:3]
grid_fr, grid_fg = grid_vxc[0:2]
grid_frr, grid_frg, grid_fgg = grid_fxc[0:3]
grid_fr  *= grid_weight
grid_fg  *= grid_weight
grid_frr *= grid_weight
grid_frg *= grid_weight
grid_fgg *= grid_weight

上面都是与原子无关，是分子本身定义给出的张量；还有一些因原子不同而不同的张量．在此，我们定义两个张量：

* `grid_A_rho_1` : $\rho_{Atg} = 2 D_{\mu_A \nu} \phi_{t g \mu_A} \phi_{g \nu}$；
* `grid_A_rho_2` : $\rho_{Atrg} = 4 D_{\mu_A \nu} (\phi_{tr \mu_A} \phi_{\nu} + \phi_{t \mu_A} \phi_{r \nu})$

In [ ]:
%%time
grid_A_rho_1 = np.zeros((natm, 3, ngrid))
grid_A_rho_2 = np.zeros((natm, 3, 3, ngrid))
for A in range(natm):
    sA = mol_slice(A)
    grid_A_rho_1[A]  = 2 * np.einsum("tgk ,  gl, kl -> tg ", grid_ao_1[:, :, sA],    grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2[A]  = 2 * np.einsum("trgk,  gl, kl -> trg", grid_ao_2[:, :, :, sA], grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2[A] += 2 * np.einsum("tgk , rgl, kl -> trg", grid_ao_1[:, :, sA],    grid_ao_1, D[sA], optimize=True)

## GGA 一阶核坐标梯度

### HF 部分

GGA 的一阶梯度中，大部分仍然是 HF 的贡献．我们在以前已经讨论过 [HF 梯度](hf_nuc_grad.ipynb)；GGA 中，至多就是在交换积分的部分乘上杂化泛函的系数．对于 B3LYP，$c_x = 0.2$．

In [ ]:
def my_grad_hf(scf_eng):
    mol = scf_eng.mol
    C = scf_eng.mo_coeff
    D = scf_eng.make_rdm1()
    mo_occ = scf_eng.mo_occ
    e = scf_eng.mo_energy
    De = np.einsum("ui, i, vi -> uv", C, e * mo_occ, C)
    c_x = scf_eng._numint.hybrid_coeff(scf_eng.xc)
    integrals  = - mol.intor("int1e_ipkin") - mol.intor("int1e_ipnuc") \
                 - np.einsum("tuvkl, kl -> tuv", mol.intor('int2e_ip1'), D) \
                 + c_x * 0.5 * np.einsum("tukvl, kl -> tuv", mol.intor('int2e_ip1'), D)
    integral_ovlp = mol.intor('int1e_ipovlp')
    grad_elec = np.zeros((mol.natm, 3))
    for A in range(mol.natm):
        sA = mol_slice(A)
        grad_elec[A, :] += np.einsum("tuv, uv -> t", integrals[:, sA], D[sA]) * 2
        grad_elec[A, :] += np.einsum("tuv, uv -> t", integral_ovlp[:, sA], De[sA]) * 2
        with mol.with_rinv_as_nucleus(A):
            grad_elec[A, :] -= np.einsum("tuv, uv -> t", mol.intor("int1e_iprinv") * mol.atom_charge(A), D) * 2
    return grad_elec

grad_hf = my_grad_hf(scf_eng)

### GGA 部分

GGA 电子态能量一阶梯度的另一个贡献的部分是交换相关能关于坐标的导数．能量导数的表达式的产生并非显然；下面简单叙述导数表达式的产生过程．

首先，根据 [泛函变分的意义](https://en.wikipedia.org/wiki/Functional_derivative#Functional_derivative)，有下式成立：

\begin{equation}
\int \frac{\delta E_\mathrm{xc}}{\rho(\boldsymbol{r})} f(\boldsymbol{r}) \, \mathrm{d} \boldsymbol{r} = \lim_{\varepsilon \rightarrow 0} \frac{E_\mathrm{xc} [\rho + \varepsilon f] - E_\mathrm{xc} [\rho]}{\varepsilon} = \left. \frac{\partial}{\partial \varepsilon} E_\mathrm{xc} [\rho + \varepsilon f] \right|_{\varepsilon \rightarrow 0}
\tag{1} \label{eq.1}
\end{equation}

其次，我们从微元法的角度指出，如果现在 $\varepsilon$ 指代原子核坐标分量的移动量 $A_t$；$\rho$ 是未被微扰的密度，而 $\rho + \varepsilon f$ 是沿着 $\varepsilon$ 所指代的方向被微扰后的电子密度，那么微扰密度将是 $f(\boldsymbol{r}) = \partial_{A_t} \rho = D_{\mu \nu} \partial_{A_t} (\phi_\mu \phi_\nu)$．我们将这个结果代入上式，则有

\begin{equation}
\left. \frac{\partial}{\partial A_t} E_\mathrm{xc} [\rho + A_t (\partial_{A_t} + \partial_{A_t}^\mathrm{U}) \rho] \right|_{A_t \rightarrow 0}
= \int \frac{\delta E_\mathrm{xc}}{\delta \rho} (\partial_{A_t} + \partial_{A_t}^\mathrm{U}) \rho \, \mathrm{d} \boldsymbol{r}
\end{equation}

我们就认为上式即是交换相关能对原子核坐标分量 $A_t$ 的导数了．

事实上，上式中的 U 矩阵相关部分已经被包含在 HF 一阶梯度的贡献了．考察 $\frac{\partial}{\partial A_t} E_\mathrm{xc} [\rho]$，U 矩阵部分的贡献 $\partial_{A_t}^\mathrm{U} E_\mathrm{xc} [\rho]$ 可以记为

\begin{equation}
\int \frac{\delta E_\mathrm{xc}}{\delta \rho} \partial_{A_t}^\mathrm{U} \rho \, \mathrm{d} \boldsymbol{r}
= 4 U_{pi}^{A_t} C_{\mu p} C_{\nu i} \int \frac{\delta E_\mathrm{xc}}{\delta \rho} (\phi_\mu \phi_\nu) \, \mathrm{d} \boldsymbol{r}
\end{equation}

在 [交换相关势的推导](xyg3_energy.ipynb#交换相关势的推导) 中，我们已经知道 $\int \delta_\rho E_\mathrm{xc} \cdot (\phi_\mu \phi_\nu) \, \mathrm{d} \boldsymbol{r}$ 即是 KS 原子轨道 Fock 矩阵的贡献成员；当其与其它 Fock 矩阵贡献成员相加后，再乘以张量 $C_{\mu p} C_{\nu i}$ 并对角标 $\mu, \nu$ 求和，即得到 $F_{pi}$．我们知道，$4 U_{pi}^{A_t} F_{pi}$ 与 $\partial_{A_t}^\mathrm{U} \big( 2 (ii|jj) - (ij|ij) \big)$ 的和，在非占-占据部分恰好是 CP-HF 方程，因此抵消为零；而在占据-占据部分，根据 $U_{ij}^{A_t} + U_{ji}^{A_t} + S_{ij}^{A_t} = 0$，可以化为 $-2 S_{ij}^{A_t} F_{ij} = -2 S_{ii}^{A_t} \varepsilon_i$．因此，交换相关能一阶梯度的 U 矩阵贡献部分其实已经包含在 HF 梯度框架里了．

因此，我们只要考虑核 U 矩阵无关部分的导数即可．其推导如下

\begin{align}
\partial_{A_t} E_\mathrm{xc}[\rho]
&= \int \left( \frac{\partial f^\mathrm{xc}}{\partial \rho} \partial_{A_t} \rho - \nabla_{\boldsymbol{r}} \cdot (\frac{\partial f^\mathrm{xc}}{\partial \nabla_{\boldsymbol{r}} \rho}) \partial_{A_t} \rho \right) \, \mathrm{d} \boldsymbol{r} \\
&= \int \left(
- \frac{\partial f^\mathrm{xc}}{\partial \rho} \rho_{At}
- \frac{\partial f^\mathrm{xc}}{\partial \nabla_{\boldsymbol{r}} \rho} \cdot \nabla_{\boldsymbol{r}} \rho_{At}
\right) \, \mathrm{d} \boldsymbol{r} \\
&= - \int \left(
\frac{\partial f^\mathrm{xc}}{\partial \rho} \rho_{At}
+ 2 \frac{\partial f^\mathrm{xc}}{\partial \gamma} \nabla_{\boldsymbol{r}} \rho \cdot \nabla_{\boldsymbol{r}} \rho_{At}
\right) \, \mathrm{d} \boldsymbol{r} \\
&= - \int \left( f^\rho \rho_{At} + 2 f^\gamma \rho_r \rho_{Atr}
\right) \, \mathrm{d} \boldsymbol{r} \\
&= - w_g f^\rho_g \rho_{Atg} - 2 w_g f^\gamma_g \rho_{rg} \rho_{Atrg}
\tag{2} \label{eq.2}
\end{align}

上式的第一个等号是变分定义；第二个等号为 $\partial_{A_t} \rho = - \rho_{At}$，同时利用了 $\nabla_{\boldsymbol{r}}$ 的反厄米性；第三个等号利用 $\partial_{\nabla_{\boldsymbol{r}} \rho} \gamma = 2 \nabla_{\boldsymbol{r}} \rho$；第四个等号是利用了内积定义．

上式实际上只有两项，因此只需要两行代码即解决 GGA 的 U 矩阵无关部分导数．我们可以立即核验 GGA 的一阶梯度：

In [ ]:
%%time
grad_gga  = -     np.einsum("g, Atg      -> At", grid_fr, grid_A_rho_1)
grad_gga += - 2 * np.einsum("g, Atrg, rg -> At", grid_fg, grid_A_rho_2, grid_rho_1)
print(np.allclose(grad_hf + grad_gga, scf_grad.grad_elec()))

我们以后还会经常会使用下述的关系，其中 $f$ 代表泛函核：

\begin{equation}
\partial_{t} f = f^\rho \cdot \partial_t \rho + 2 f^\gamma \rho_r \cdot \partial_t \rho_r
\tag{3} \label{eq.3}
\end{equation}

以后遇到这种情况时，将会不加声明地简化记号．

<div class="alert alert-info">

**提示**

对 $\rho (\boldsymbol{r})$ 关于 $\boldsymbol{r}$ 的偏导数不会产生 U 矩阵相关项，因为 $\boldsymbol{r}$ 是电子坐标，分子轨道系数 $C_{\mu p}$ 对其偏导数为零；那么自然地，$\partial_{r}^\mathrm{U}$ 是没有意义的．但是，如果 $t$ 是外部性质，譬如原子核坐标或电场，那么对 $\rho$ 的关于 $t$ 的偏导数应为 $(\partial_{t} + \partial_{t}^\mathrm{U}) \rho$．

</div>

## GGA 二阶核坐标梯度

### U 矩阵无关部分

我们首先会简单推导 GGA 二阶梯度的公式，以此为基础写代码．GGA 二阶梯度的 U 矩阵无关部分相比于 HF 二阶梯度的 U 矩阵无关部分，多出的项是 $\partial_{B_s} \partial_{A_t} E_\mathrm{xc}[\rho]$．由于 $\partial_{A_t} E_\mathrm{xc}[\rho]$ 在 [(2)](#mjx-eqn-eq.2) 中已经求得，我们只需要使用偏导数的求导法则就可以解决 $\partial_{B_s} \partial_{A_t} E_\mathrm{xc}[\rho]$，不必要使用复杂的二阶变分．

\begin{align}
\partial_{B_s} \partial_{A_t} E_\mathrm{xc}[\rho] &= 
- \int \partial_{s_B} \left( 
f^\rho \rho_{At} + 2 f^\gamma \rho_r \rho_{Atr}
\right) \, \mathrm{d} \boldsymbol{r} \\
&= - \int \left(
(f^{\rho \rho} \partial_{s_B} \rho + 2 f^{\rho \gamma} \rho_w \partial_{s_B} \rho_w) \rho_{At} + f^\rho \partial_{s_B} \rho_{At}
\right) \, \mathrm{d} \boldsymbol{r} \\
&\quad - 2 \int \left(
(f^{\rho \gamma} \partial_{s_B} \rho + 2 f^{\gamma \gamma} \rho_w \partial_{s_B} \rho_w) \rho_r \rho_{Atr} + f^\gamma \rho_r \partial_{s_B} \rho_{Atr} + f^\gamma (\partial_{s_B} \rho_r) \rho_{Atr}
\right) \, \mathrm{d} \boldsymbol{r} \\
&= \int \left(
f^{\rho \rho} \rho_{At} \rho_{Bs} + 2 f^{\rho \gamma} \rho_{At} \rho_{Bsw} \rho_w + 2 f^{\rho \gamma} \rho_{Atr} \rho_{Bs} \rho_{r} + 4 f^{\gamma \gamma} \rho_{Atr} \rho_{Bsw} \rho_r \rho_w + 2 f^{\gamma} \rho_{Atr} \rho_{Bsr}
\right) \, \mathrm{d} \boldsymbol{r}
\tag{4.1} \label{eq.4.1} \\
&\quad + D_{\mu_A \nu_B} \int \left(
2 f^\rho \phi_{t \mu_A} \phi_{s \nu_B} + 4 f^\gamma \rho_r \phi_{t \mu_A} \phi_{sr \nu_B} + 4 f^\gamma \rho_r \phi_{tr \mu_A} \phi_{s \nu_B}
\right) \, \mathrm{d} \boldsymbol{r}
\tag{4.2} \label{eq.4.2} \\
&\quad + D_{\mu_{AB} \nu} \int \left(
2 f^\rho \phi_{ts \mu_{AB}} \phi_\nu + 4 f^\gamma \rho_r \phi_{ts \mu_{AB}} \phi_{r \nu} + 4 f^\gamma \rho_r \phi_{tsr \mu_{AB}} \phi_\nu
\right) \, \mathrm{d} \boldsymbol{r}
\tag{4.3} \label{eq.4.3}
\end{align}

上式中每个等号均是简单的拆项，以及利用 $\partial_{s_B} \rho = - \rho_{Bs}$ 等关系式．

式 [(4.1)](#mjx-eqn-eq.4.1) 可以直接展开为格点求和的形式：

\begin{equation}
w_g ( f^{\rho \rho}_g \rho_{Atg} \rho_{Bsg} + 2 f^{\rho \gamma}_g \rho_{Atg} \rho_{Bswg} \rho_{wg} + 2 f^{\rho \gamma}_g \rho_{Atrg} \rho_{Bsg} \rho_{rg} + 4 f^{\gamma \gamma}_g \rho_{Atrg} \rho_{Bswg} \rho_{rg} \rho_{wg} + 2 f^{\gamma}_g \rho_{Atrg} \rho_{Bsrg} )
\end{equation}

In [ ]:
%%time
hess_noU_gga_1  = 1 * np.einsum("g, Atg , Bsg          -> ABts", grid_frr, grid_A_rho_1, grid_A_rho_1,                         optimize=True)
hess_noU_gga_1 += 2 * np.einsum("g, Atg , Bswg, wg     -> ABts", grid_frg, grid_A_rho_1, grid_A_rho_2, grid_rho_1,             optimize=True)
hess_noU_gga_1 += 2 * np.einsum("g, Atrg, Bsg , rg     -> ABts", grid_frg, grid_A_rho_2, grid_A_rho_1, grid_rho_1,             optimize=True)
hess_noU_gga_1 += 4 * np.einsum("g, Atrg, Bswg, rg, wg -> ABts", grid_fgg, grid_A_rho_2, grid_A_rho_2, grid_rho_1, grid_rho_1, optimize=True)
hess_noU_gga_1 += 2 * np.einsum("g, Atrg, Bsrg         -> ABts", grid_fg,  grid_A_rho_2, grid_A_rho_2,                         optimize=True)

对于式 [(4.2)](#mjx-eqn-eq.4.2)，我们首先需要通过格点求和，写出关于 $t, s, \mu, \nu$ 的张量；随后与密度矩阵在特定原子的 slice 下相乘求和得到结果．格点求和后的张量为 `hess_noU_gga_2_ao`，其形式是

\begin{equation}
w_g ( 2 f^\rho_g \phi_{tg \mu} \phi_{sg \nu} + 4 f^\gamma_g \rho_{rg} \phi_{tg \mu} \phi_{srg \nu} + 4 f^\gamma_g\rho_{rg} \phi_{trg \mu} \phi_{sg \nu} )
\end{equation}

注意到上式中，$4 f^\gamma_g \rho_{rg} \phi_{tg \mu} \phi_{srg \nu}$ 与 $4 f^\gamma_g\rho_{rg} \phi_{trg \mu} \phi_{sg \nu}$ 在同时互换 $t, s$ 和 $\mu, \nu$ 后是等价的，因此可以节省这部分计算量，通过张量转置来赋值．

In [ ]:
%%time
hess_noU_gga_2 = np.zeros((natm, natm, 3, 3))
hess_noU_gga_2_ao  = 4 * np.einsum("g, rg, trgu, sgv -> tsuv", grid_fg, grid_rho_1, grid_ao_2, grid_ao_1, optimize=True)
hess_noU_gga_2_ao += hess_noU_gga_2_ao.transpose(1, 0, 3, 2)
hess_noU_gga_2_ao += 2 * np.einsum("g, tgu, sgv -> tsuv", grid_fr, grid_ao_1, grid_ao_1, optimize=True)

for A in range(natm):
    for B in range(A, natm):
        sA, sB = mol_slice(A), mol_slice(B)
        hess_noU_gga_2[A, B] += np.einsum("tsuv, uv -> ts", hess_noU_gga_2_ao[:, :, sA, sB], D[sA, sB])
        hess_noU_gga_2[B, A] = hess_noU_gga_2[A, B].T

在 PySCF 中，上面两个 Hessian 贡献项可以通过 `hessian.rks._get_vxc_deriv2` 计算给出．由于我们的代码中，[(4.1)](#mjx-eqn-eq.4.1) 的计算时不生成原子轨道矩阵，因此我们直接验证两式对 Hessian 的贡献：

In [ ]:
%%time
vxc_deriv2_pyscf = hessian.rks._get_vxc_deriv2(scf_hess, C, mo_occ, 2000)
hess_deriv2_pyscf = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    for B in range(natm):
        sA, sB = mol_slice(A), mol_slice(B)
        hess_deriv2_pyscf[A, B] = np.einsum("tsuv, uv -> ts", vxc_deriv2_pyscf[A, :, :, sB], D[sB]) * 2

In [ ]:
print(np.allclose(hess_noU_gga_1 + hess_noU_gga_2, hess_deriv2_pyscf))

不过在 PySCF 中，[(4.1)](#mjx-eqn-eq.4.1) 的计算过程中生成了显含格点的中间张量，在内存 I/O 上有所拖累；因此计算耗时相对较长．

[(4.3)](#mjx-eqn-eq.4.3) 与 [(4.2)](#mjx-eqn-eq.4.2) 类似，需要先生成原子轨道大小的矩阵，随后与密度矩阵相乘；只是在这里，$t, s$ 分量总是同时被求导；因此，我们可以用 $T = ts$ 的来表示这些张量．格点求和后的张量为

$$
2 f^\rho_g \phi_{Tg \mu} \phi_{g \nu} + 4 f^\gamma_g \rho_{rg} \phi_{Tg \mu} \phi_{rg \nu} + 4 f^\gamma_g \rho_{rg} \phi_{rTg \mu} \phi_{g \nu}
$$

In [ ]:
%%time
hess_noU_gga_3_ao  = 2 * np.einsum("g,      Tgu,  gv -> Tuv", grid_fr, grid_ao_2T, grid_ao_0, optimize=True)
hess_noU_gga_3_ao += 4 * np.einsum("g, rg,  Tgu, rgv -> Tuv", grid_fg, grid_rho_1, grid_ao_2T, grid_ao_1, optimize=True)
hess_noU_gga_3_ao += 4 * np.einsum("g, rg, rTgu,  gv -> Tuv", grid_fg, grid_rho_1, grid_ao_3T, grid_ao_0, optimize=True)

In [ ]:
%%time
XX, XY, XZ, YY, YZ, ZZ = range(6)
hess_noU_gga_3 = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    sA = mol_slice(A)
    hess_noU_gga_3[A, A] = np.einsum("Tuv, uv -> T", hess_noU_gga_3_ao[:, sA], D[sA])[[XX, XY, XZ, XY, YY, YZ, XZ, YZ, ZZ]].reshape((3, 3))

在 PySCF 中，格点求和后的张量可以通过下面的代码生成；我们可以以此验证我们所生成的张量：

In [ ]:
%%time
vxc_diag_pyscf = hessian.rks._get_vxc_diag(scf_hess, C, mo_occ, 2000)
print(np.allclose(vxc_diag_pyscf, hess_noU_gga_3_ao[[XX, XY, XZ, XY, YY, YZ, XZ, YZ, ZZ]].reshape((3, 3, nao, nao)) / 2))

至此，我们已经完整地生成了所有的 U 矩阵无关的 GGA 部分的 Hessian 贡献项．与 HF 部分的结果相加之后，就完成了总的 U 矩阵无关的 Hessian 贡献计算．HF 部分的计算过程代码参见如下；这几乎是从 [my_rhf_hess.py](include/my_rhf_hess.py) 中复制来的；除了在交换积分贡献上乘上了杂化泛函系数 $c_x$．

In [ ]:
%%time
def get_hess_ao_noU_hcore(A, B):
    ao_matrix = np.zeros((3 * 3, nao, nao))
    zA, zB = mol.atom_charge(A), mol.atom_charge(B)
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += int1e_ipipkin[:, sA]
        ao_matrix[:, sA] += int1e_ipipnuc[:, sA]
        with mol.with_rinv_as_nucleus(A):
            ao_matrix -= zA * mol.intor('int1e_ipiprinv')
            ao_matrix -= zA * mol.intor('int1e_iprinvip')
    ao_matrix[:, sA, sB] += int1e_ipkinip[:, sA, sB]
    ao_matrix[:, sA, sB] += int1e_ipnucip[:, sA, sB]
    with mol.with_rinv_as_nucleus(B):
        ao_matrix[:, sA] += zB * mol.intor('int1e_ipiprinv')[:, sA]
        ao_matrix[:, sA] += zB * mol.intor('int1e_iprinvip')[:, sA]
    with mol.with_rinv_as_nucleus(A):
        ao_matrix[:, sB] += zA * mol.intor('int1e_ipiprinv')[:, sB]
        ao_matrix[:, sB] += zA * mol.intor('int1e_iprinvip').swapaxes(1, 2)[:, sB]
    ao_matrix += ao_matrix.swapaxes(1, 2)
    return ao_matrix

eri_mat1 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipip1, D) * 2 - c_x * np.einsum("Tukvl, kl -> Tuv", int2e_ipip1, D)
eri_mat2 = np.einsum("Tuvkl, kl -> Tuv", int2e_ipvip1, D) * 2 - c_x * np.einsum("Tukvl, kl -> Tuv", int2e_ip1ip2, D)
eri_tensor1 = int2e_ip1ip2 * 4 - c_x * int2e_ipvip1.swapaxes(2, 3) - c_x * int2e_ip1ip2.swapaxes(2, 4)

def get_hess_ao_noU_eri(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] += eri_mat1[:, sA]
    ao_matrix[:, sA, sB] += eri_mat2[:, sA, sB]
    ao_matrix[:, sA] += np.einsum("Tuvkl, kl -> Tuv", eri_tensor1[:, sA, :, sB], D[sB])
    return ao_matrix

def get_hess_ao_noU_S(A, B):
    ao_matrix = np.zeros((9, nao, nao))
    sA, sB = mol_slice(A), mol_slice(B)
    if (A == B):
        ao_matrix[:, sA] -= int1e_ipipovlp[:, sA] * 2
    ao_matrix[:, sA, sB] -= int1e_ipovlpip[:, sA, sB] * 2
    return ao_matrix

def get_hess_noU(A, B):
    return (np.einsum("Tuv, uv -> T", get_hess_ao_noU_hcore(A, B) + get_hess_ao_noU_eri(A, B), D).reshape(3, 3)
            + np.einsum("Tuv, uv -> T",  + get_hess_ao_noU_S(A, B), De).reshape(3, 3))

hess_noU_noGGA = np.array([ [ get_hess_noU(A, B) for B in range(natm) ] for A in range(natm) ])

我们最终生成的 U 矩阵无关部分的比对如下：

In [ ]:
%%time
partial_hess_pyscf = scf_hess.partial_hess_elec()

In [ ]:
np.allclose(
    hess_noU_noGGA + hess_noU_gga_1 + hess_noU_gga_2 + hess_noU_gga_3,
    partial_hess_pyscf
)

### U 矩阵相关部分

在这一部分中，U 矩阵相关部分指代 GGA 泛函一阶导 $\partial_{A_t} E_\mathrm{xc}[\rho]$ 在另一个坐标下 $B_s$ 的导数的 U 矩阵贡献 $\partial_{B_s}^\mathrm{U} \partial_{A_t} E_\mathrm{xc}[\rho]$．这部分的贡献会加到 $U_{pi}^{B_s} F_{pi}^{A_t}$ 中，并且会通过 $F_{pi}^{A_t}$ 改变 CP-HF 方程，成为 CP-KS 方程．

\begin{align}
\partial_{B_s}^\mathrm{U} \partial_{A_t} E_\mathrm{xc}[\rho] 
&= - \int \left(
(f^{\rho \rho} \partial_{s_B}^\mathrm{U} \rho + 2 f^{\rho \gamma} \rho_w \partial_{s_B}^\mathrm{U} \rho_w) \rho_{At} + f^\rho \partial_{s_B}^\mathrm{U} \rho_{At}
\right) \, \mathrm{d} \boldsymbol{r} \\
&\quad - 2 \int \left(
(f^{\rho \gamma} \partial_{s_B}^\mathrm{U} \rho + 2 f^{\gamma \gamma} \rho_w \partial_{s_B}^\mathrm{U} \rho_w) \rho_r \rho_{Atr} + f^\gamma \rho_r \partial_{s_B}^\mathrm{U} \rho_{Atr} + f^\gamma (\partial_{s_B}^\mathrm{U} \rho_r) \rho_{Atr}
\right) \, \mathrm{d} \boldsymbol{r} \\
\end{align}

如果我们不考虑出现二阶 U 矩阵的情况，而只将一阶 U 矩阵的贡献写出，那么就有下式：

\begin{align}
\partial_{B_s}^\mathrm{U} \partial_{A_t} E_\mathrm{xc}[\rho]^{(1)}
&= - 4 U_{pi}^{B_t} C_{\mu p} C_{\nu i} \int \big(
f^{\rho \rho} \rho_{At} (\phi_\mu \phi_\nu)
+ 2 f^{\rho \gamma} \rho_{At} \rho_w \partial_{w} (\phi_\mu \phi_\nu)
\\ &\qquad\quad
+ 2 f^{\rho \gamma} \rho_{Atr} \rho_r (\phi_\mu \phi_\nu)
+ 2 f^{\gamma \gamma} \rho_{Atr} \rho_w \rho_r \partial_w (\phi_\mu \phi_\nu)
+ 2 f^{\gamma} \rho_{Atr} \partial_r (\phi_\mu \phi_\nu)
\big) \, \mathrm{d} \boldsymbol{r}
\\ &\quad - 4 U_{pi}^{B_t} C_{\mu p} C_{\nu i} \int \big(
f^\rho (\phi_{t \mu_A} \phi_{\nu} + \phi_{\mu} \phi_{t \nu_A})
\\ &\qquad\quad
+ 2 f^{\gamma} \rho_r (\phi_{tr \mu_A} \phi_{\nu} + \phi_{t \mu_A} \phi_{r \nu} + \phi_{\mu} \phi_{tr \nu_A} + \phi_{r \mu} \phi_{t \nu_A})
\big) \, \mathrm{d} \boldsymbol{r}
\end{align}

这里我们指出，类似于一阶 GGA 梯度，二阶 GGA 梯度中所有出现的二阶 U 矩阵的贡献可以通过二阶 CP-KS 方程化为项 $- 2 S_{ii}^{A_t B_s} \varepsilon_i - 2 \eta_{ii}^{A_t B_s} \varepsilon_i$．这些项的计算或者已经在 U 矩阵无关的二阶梯度中，或者通过一阶 U 矩阵乘积就可以获得．

注意到，如果我们将上式中的 $U_{pi}^{B_t} C_{\mu p} C_{\nu i}$ 抽出，将剩下的项归并为 $F_{\mu \nu}^{\mathrm{GGA}, A_t}$；那么我们会发现，$F_{\mu \nu}^{\mathrm{GGA}, A_t}$ 必须是对称矩阵，不可以像之前一样交换 $\mu$ 与 $\nu$ 后化简．

\begin{equation}
\partial_{B_s}^\mathrm{U} \partial_{A_t} E_\mathrm{xc}[\rho]^{(1)} = 4 U_{pi}^{B_t} C_{\mu p} C_{\nu i} F_{\mu \nu}^{\mathrm{GGA}, A_t}
\end{equation}

\begin{align}
F_{\mu \nu}^{\mathrm{GGA}, A_t}
&= - w_g (\frac{1}{2} f^{\rho}_g \rho_{Atg} \phi_{g\mu} \phi_{g\nu} + 2 f^{\rho\gamma}_g \rho_{Atg} \rho_{wg} \phi_{wg\mu} \phi_{g\nu}
\\ &\qquad\quad
+ f^{\rho\gamma}_g \rho_{rg} \rho_{Atrg} \phi_{g\mu} \phi_{g\nu} + 2 \rho_{rg} \rho_{Atrg} \rho_{wg} \rho_{wg\mu} \rho_{g\nu} + 2 f^\gamma_g \rho_{Atrg} \phi_{rg\mu} \phi_{g\nu}
) \\ &\quad
- w_g (f^\rho_g \phi_{tg\mu_A} \phi_{g\nu} + 2 f^\gamma_g \phi_{trg\mu_A} \phi_{g\nu} + 2 f^\gamma_g \phi_{tg\mu_A} \phi_{r\nu})
\\ &\quad
+ \mathrm{interchange} (\mu, \nu)
\end{align}

上式可以分为三部分：第一部分中，$A$ 在密度记号下；第二部分下，$A$ 则在轨道记号下；第三部分则是将矩阵对称化．

第一部分的表达式可以用代码写为：

In [ ]:
EINOPT = ["greedy", 1024**3 * 2 / 8]

In [ ]:
%%time
hess_ao_hgga  = -0.5 * np.einsum("g,     Atg ,      gu, gv -> Atuv", grid_frr,               grid_A_rho_1,             grid_ao_0, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -  2 * np.einsum("g,     Atg , wg, wgu, gv -> Atuv", grid_frg,               grid_A_rho_1, grid_rho_1, grid_ao_1, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -      np.einsum("g, rg, Atrg,      gu, gv -> Atuv", grid_frg, grid_rho_1,   grid_A_rho_2,             grid_ao_0, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -  4 * np.einsum("g, rg, Atrg, wg, wgu, gv -> Atuv", grid_fgg, grid_rho_1,   grid_A_rho_2, grid_rho_1, grid_ao_1, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -  2 * np.einsum("g,     Atrg,     rgu, gv -> Atuv", grid_fg,                grid_A_rho_2,             grid_ao_1, grid_ao_0, optimize=EINOPT)

第二部分的表达式，由于表达式中显含轨道和原子标号之间的关系，需要直接地使用循环：

In [ ]:
%%time
hess_ao_hgga_mat1  = -   np.einsum("g,      tgu,  gv -> tuv", grid_fr,             grid_ao_1, grid_ao_0, optimize=EINOPT)
hess_ao_hgga_mat1 -= 2 * np.einsum("g, rg,  tgu, rgv -> tuv", grid_fg, grid_rho_1, grid_ao_1, grid_ao_1, optimize=EINOPT)
hess_ao_hgga_mat1 -= 2 * np.einsum("g, rg, trgu,  gv -> tuv", grid_fg, grid_rho_1, grid_ao_2, grid_ao_0, optimize=EINOPT)

for A in range(natm):
    sA = mol_slice(A)
    hess_ao_hgga[A, :, sA] += hess_ao_hgga_mat1[:, sA]

第三部分则是简单的对称化：

In [ ]:
%%time
hess_ao_hgga += hess_ao_hgga.swapaxes(2, 3)

在 PySCF 中，存在函数生成该矩阵．我们可以对照这两者的结果：

In [ ]:
%%time
vxc_deriv1_pyscf = hessian.rks._get_vxc_deriv1(scf_hess, C, mo_occ, 2000)
print(np.allclose(vxc_deriv1_pyscf, hess_ao_hgga))

至于上述代码中出现的 `EINOPT`，它会将可用内存大小传入 `np.einsum`，以允许计算量小但内存占用大的张量缩并计算．详细的讨论会在最后一小节．

### CP-KS 方程的 $A_{pi, qj}$ 贡献

我们知道 CP-HF 方程的导出是从 $(\partial_{A_t} + \partial_{A_t}^\mathrm{U}) F_{ai} = 0$ 而产生的．对于 GGA 而言，其相对于 HF 方法中的 $F_{ai}$ 多出了 GGA 所特有的对 Fock 矩阵的贡献项．因此，CP-KS 方程中，也多出了这部分的贡献．

在刚才，我们将 $\partial_{B_s}^\mathrm{U} \partial_{A_t} E_\mathrm{xc}[\rho]$ 抽出 $4 U_{pi}^{B_t} C_{\mu p} C_{\nu i}$ 命名为 $F_{\mu \nu}^{\mathrm{GGA}, A_t}$；但导出 $F_{\mu \nu}^{\mathrm{GGA}, A_t}$ 还有另一种方法，即 $F_{\mu \nu}^{\mathrm{GGA}, A_t} = \partial_{A_t} F_{\mu \nu}^\mathrm{GGA}$，若

\begin{equation}
F_{\mu \nu}^\mathrm{GGA} = \int \frac{\delta E_\mathrm{xc}}{\delta \rho} \phi_{\mu} \phi_{\nu} \, \mathrm{d} \boldsymbol{r}
\end{equation}

这可以留作练习．但这里指出，$\partial_{A_t}^\mathrm{U} F_{\mu \nu}^\mathrm{GGA}$ 在推导过程中与 $\partial_{B_s}^\mathrm{U} \partial_{A_t} F_{\mu \nu}^\mathrm{GGA}$ 的推导很相似，因为 $F_{\mu \nu}^\mathrm{GGA}$ 形式上与 $\partial_{A_t} E^\mathrm{xc}$ 几乎一样；但需要注意被求导对象的密度到底是不是自洽场密度；这里定义广义密度 $R_{\mu \nu}$ 与 $\varrho = R_{\mu \nu} \phi_\mu \phi_{\nu}$，在求导时，我们不考虑广义密度的导数．这里就直接写出结果：

\begin{align}
\partial_{A_t}^\mathrm{U} F_{\kappa \lambda}^\mathrm{GGA} R_{\kappa \lambda}
&= \int \partial_{A_t}^\mathrm{U} (f^\rho_g \varrho + 2 f^\gamma \rho_r \varrho_r) \, \mathrm{d} \boldsymbol{r} \\
&= 4 U_{pi} C_{\mu p} C_{\nu i} w_g \big(
\\ &\qquad
\frac{1}{2} f^{\rho \rho}_g \varrho_g \phi_{g \mu} \phi_{g \nu}
+ 2 f^{\rho \gamma} \varrho_g \rho_r \phi_{rg \mu} \phi_{g \nu}
\\ &\qquad
+ f^{\rho \gamma} \rho_{rg} \varrho_{rg} \phi_{g \mu} \phi_{g \nu}
+ 4 f^{\gamma \gamma} \rho_{rg} \varrho_{rg} \phi_{wg \mu} \phi_{g \nu}
+ 2 f^{\gamma} \varrho_{rg} \phi_{rg \mu} \phi_{g \nu}
\\ &\qquad
+ \mathrm{interchange} (\mu, \nu)
\big)
\end{align}

我们将上述结果去除 $U_{pi} C_{\mu p} C_{\nu i}$，剩下的部分就可以加到 HF 部分，构成完整的 GGA 的 $A_{pi, qj} R_{qj}$．代码上的解释如下：

In [ ]:
def Ax(x):
    shape = x.shape
    x = x.reshape((-1, nmo, nocc))
    dx = C @ x @ Co.T
    v = np.zeros_like(x)
    for i in range(dx.shape[0]):
        dm = dx[i] + dx[i].T
        v[i] = C.T @ (scf_eng.get_j(mol, dm) - 0.5 * c_x * scf_eng.get_k(mol, dm)) @ Co
        # GGA Part
        grid_rho_dm_0 = np.einsum("uv, gu, gv -> g", dm, grid_ao_0, grid_ao_0, optimize=True)
        grid_rho_dm_1 = np.einsum("uv, tgu, gv -> tg", dm, grid_ao_1, grid_ao_0, optimize=True) * 2
        vks  = 0.5 * np.einsum("g,      g,      gu, gv -> uv", grid_frr,               grid_rho_dm_0,             grid_ao_0, grid_ao_0, optimize=True)
        vks +=   2 * np.einsum("g,      g, wg, wgu, gv -> uv", grid_frg,               grid_rho_dm_0, grid_rho_1, grid_ao_1, grid_ao_0, optimize=True)
        vks +=       np.einsum("g, rg, rg,      gu, gv -> uv", grid_frg, grid_rho_1,   grid_rho_dm_1,             grid_ao_0, grid_ao_0, optimize=True)
        vks +=   4 * np.einsum("g, rg, rg, wg, wgu, gv -> uv", grid_fgg, grid_rho_1,   grid_rho_dm_1, grid_rho_1, grid_ao_1, grid_ao_0, optimize=True)
        vks +=   2 * np.einsum("g,     rg,     rgu, gv -> uv", grid_fg,                grid_rho_dm_1,             grid_ao_1, grid_ao_0, optimize=True)
        vks += vks.T
        v[i] += C.T @ vks @ Co
    return 2 * v.reshape(shape)

PySCF 中，相同的函数在 RHF 的框架内定义．我们可以拿一个随机的密度矩阵来作比对：

In [ ]:
rand_pi = np.random.random((nmo, nocc))

In [ ]:
%%time
rand_pi_pyscf = hessian.rhf.gen_vind(scf_eng, C, mo_occ)(rand_pi)

In [ ]:
%%time
rand_pi_my = Ax(rand_pi)

In [ ]:
np.allclose(rand_pi_pyscf, rand_pi_my)

但是需要注意到，一方面，PySCF 的速度看上去也许偏慢，但由于 PySCF 库函数本身的调用 (以及不少我不清楚的原因)，PySCF 的代码在真正执行 CP-KS 方程时，`hessian.rhf.gen_vind` 的调用速度会快不少；另一方面，CP-KS 会大量调用 `Ax` 函数．因此，`Ax` 的计算效率最好要优化到最好，否则及其拖累 CP-KS 方程求解速度．

下面就是我简单优化后的 `Ax` 代码．首先，我们不在 `Ax` 中现时生成格点密度，而将所有格点计算放在全局空间中．这就允许 `Ax` 中只需要调入基组四次方 (在体系和基组较小时，格点的影响重要得多，未优化代码会被格点数量严重拖累；但在大基组，譬如目前 (99, 590) 格点的情况下，原子轨道数大于 361 时，基组四次方劣于格点乘基组平方，优化后的代码速度反而会被基组大小拖累)．但优化的代价是全局空间中我们会计算原子轨道四次方大小的张量 `rdm2_inAx`，其计算复杂度是格点数乘基组的四次方．这个计算耗时不能忽视．因此，在大分子或大基组下，下面的方法未必更好．

In [ ]:
%%time
grid_rho1phi1 = np.einsum("tg, tgu -> gu", grid_rho_1, grid_ao_1, optimize=True)
rdm2_inAx  =   2 * np.einsum("g,  gk,  gu -> gku", grid_frg, grid_ao_0,     grid_rho1phi1, optimize=EINOPT)
rdm2_inAx += rdm2_inAx.swapaxes(1, 2)
rdm2_inAx += 0.5 * np.einsum("g,  gk,  gu -> gku", grid_frr, grid_ao_0,     grid_ao_0,     optimize=EINOPT)
rdm2_inAx +=   8 * np.einsum("g,  gk,  gu -> gku", grid_fgg, grid_rho1phi1, grid_rho1phi1, optimize=EINOPT)
rdm2_inAx +=   4 * np.einsum("g, rgk, rgu -> gku", grid_fg,  grid_ao_1,     grid_ao_1,     optimize=EINOPT)
rdm2_inAx = np.einsum("gku, gl, gv -> kluv", rdm2_inAx, grid_ao_0, grid_ao_0, optimize=EINOPT)

In [ ]:
def Ax(x):
    shape = x.shape
    x = x.reshape((-1, nmo, nocc))
    dx = C @ x @ Co.T
    v = np.zeros_like(x)
    for i in range(dx.shape[0]):
        dm = dx[i] + dx[i].T
        v[i] = C.T @ (scf_eng.get_j(mol, dm) - 0.5 * c_x * scf_eng.get_k(mol, dm)) @ Co
        vks = np.einsum("kluv, kl -> uv", rdm2_inAx, dm)
        v[i] += C.T @ (vks + vks.T) @ Co
    return 2 * v.reshape(shape)

我们仍然能验证现在构造出来的 `Ax` 与 PySCF 中构造的结果是相等的：

In [ ]:
%%time
rand_pi_my = Ax(rand_pi)

In [ ]:
np.allclose(rand_pi_pyscf, rand_pi_my)

### 最终结果

现在我们已经求得所有 GGA 相比于 HF 所多出的贡献项了．对于剩下的收尾工作，我们可以参考 [HF 部分的代码](include/my_rhf_hess.py)：

In [ ]:
%%time
def get_hess_ao_h1(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = (- int1e_ipkin - int1e_ipnuc 
                        - np.einsum("tuvkl, kl -> tuv", int2e_ip1, D)
                        + 0.5 * c_x * np.einsum("tukvl, kl -> tuv", int2e_ip1, D)
                       )[:, sA]
    ao_matrix -= np.einsum("tkluv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    ao_matrix += 0.5 * c_x * np.einsum("tkulv, kl -> tuv", int2e_ip1[:, sA], D[sA])
    with mol.with_rinv_as_nucleus(A):
        ao_matrix -= mol.intor("int1e_iprinv") * mol.atom_charge(A)
    return ao_matrix + ao_matrix.swapaxes(1, 2)

def get_hess_ao_s1(A):
    ao_matrix = np.zeros((3, nao, nao))
    sA = mol_slice(A)
    ao_matrix[:, sA] = -int1e_ipovlp[:, sA]
    return ao_matrix + ao_matrix.swapaxes(1, 2)

hess_ao_h1 = np.array([ get_hess_ao_h1(A) + vxc_deriv1_pyscf[A] for A in range(natm) ])
hess_ao_s1 = np.array([ get_hess_ao_s1(A) for A in range(natm) ])
hess_pi_h1 = np.einsum("Atuv, up, vi -> Atpi", hess_ao_h1, C, Co)
hess_pi_s1 = np.einsum("Atuv, up, vi -> Atpi", hess_ao_s1, C, Co)

In [ ]:
%%time
hess_U, hess_M = scf.cphf.solve(Ax, e, mo_occ, hess_pi_h1.reshape(-1, nmo, nocc), hess_pi_s1.reshape(-1, nmo, nocc))
hess_U.shape = (natm, 3, nmo, nocc); hess_M.shape = (natm, 3, nocc, nocc)

In [ ]:
%%time
hess_withU = 4 * np.einsum("Bspi, Atpi -> ABts", hess_U, hess_pi_h1)
hess_withU -= 4 * np.einsum("Bspi, Atpi, i -> ABts", hess_U, hess_pi_s1, eo)
hess_withU -= 2 * np.einsum("Atki, Bski -> ABts", hess_pi_s1[:, :, :nocc], hess_M)

最终计算得到的 Hessian 张量如下：

In [ ]:
%%time
hess_my_GGA_elec = hess_withU + hess_noU_noGGA + hess_noU_gga_1 + hess_noU_gga_2 + hess_noU_gga_3
hess_pyscf_GGA_elec = scf_hess.hess_elec()
print(np.allclose(hess_my_GGA_elec, hess_pyscf_GGA_elec))

### 二阶 GGA 梯度：代码整理

事实上，由于一些对于小分子体系的代码优化上的原因，尽管一些代码的计算效率偏低，但还有不少关键代码的计算耗时较 PySCF 本身要低，因此这里整理之后的二阶 GGA 梯度代码速度应该会更快一些．

In [ ]:
%%time

EINOPT = ["greedy", 1024**3 * 2 / 8]

# rks
c_x = scf_eng._numint.hybrid_coeff(scf_eng.xc)

nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]
mo_occ = scf_eng.mo_occ

D = scf_eng.make_rdm1()
De = np.einsum("ui, i, vi -> uv", C, e * mo_occ, C)

# grad-contrib
int1e_ipovlp = mol.intor('int1e_ipovlp')
int1e_ipkin = mol.intor("int1e_ipkin")
int1e_ipnuc = mol.intor("int1e_ipnuc")
int2e_ip1 = mol.intor("int2e_ip1")

# hess-contrib
int1e_ipipkin = mol.intor("int1e_ipipkin")
int1e_ipkinip = mol.intor("int1e_ipkinip")
int1e_ipipnuc = mol.intor("int1e_ipipnuc")
int1e_ipnucip = mol.intor("int1e_ipnucip")
int2e_ipip1 = mol.intor("int2e_ipip1")
int2e_ipvip1 = mol.intor("int2e_ipvip1")
int2e_ip1ip2 = mol.intor("int2e_ip1ip2")
int1e_ipipovlp = mol.intor("int1e_ipipovlp")
int1e_ipovlpip = mol.intor("int1e_ipovlpip")

# grid
grid_ao, grid_mask, grid_weight, grid_coord = next(scf_eng._numint.block_loop(mol, scf_eng.grids, nao, 3, 2000))
ngrid = grid_ao.shape[1]

grid_ao_0  = grid_ao[0]
grid_ao_1  = grid_ao[1:4]
grid_ao_2T = grid_ao[4:10]
XX, XY, XZ, YY, YZ, ZZ = range(4, 10)
XXX, XXY, XXZ, XYY, XYZ, XZZ, YYY, YYZ, YZZ, ZZZ = range(10, 20)

grid_ao_2 = np.array([
    [grid_ao[XX], grid_ao[XY], grid_ao[XZ]],
    [grid_ao[XY], grid_ao[YY], grid_ao[YZ]],
    [grid_ao[XZ], grid_ao[YZ], grid_ao[ZZ]],
])
grid_ao_3T = np.array([
    [grid_ao[XXX], grid_ao[XXY], grid_ao[XXZ], grid_ao[XYY], grid_ao[XYZ], grid_ao[XZZ]],
    [grid_ao[XXY], grid_ao[XYY], grid_ao[XYZ], grid_ao[YYY], grid_ao[YYZ], grid_ao[YZZ]],
    [grid_ao[XXZ], grid_ao[XYZ], grid_ao[XZZ], grid_ao[YYZ], grid_ao[YZZ], grid_ao[ZZZ]],
])

grid_rho = np.einsum("uv, tgu, gv -> tg", D, grid_ao[0:4], grid_ao_0, optimize=True)
grid_rho[1:] *= 2
grid_rho_0 = grid_rho[0]
grid_rho_1 = grid_rho[1:4]

grid_vxc, grid_fxc = scf_eng._numint.eval_xc(scf_eng.xc, grid_rho, deriv=2)[1:3]
grid_fr, grid_fg = grid_vxc[0:2]
grid_frr, grid_frg, grid_fgg = grid_fxc[0:3]
grid_fr  *= grid_weight
grid_fg  *= grid_weight
grid_frr *= grid_weight
grid_frg *= grid_weight
grid_fgg *= grid_weight

grid_A_rho_1 = np.zeros((natm, 3, ngrid))
grid_A_rho_2 = np.zeros((natm, 3, 3, ngrid))
for A in range(natm):
    sA = mol_slice(A)
    grid_A_rho_1[A]  = 2 * np.einsum("tgk ,  gl, kl -> tg ", grid_ao_1[:, :, sA],    grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2[A]  = 2 * np.einsum("trgk,  gl, kl -> trg", grid_ao_2[:, :, :, sA], grid_ao_0, D[sA], optimize=True)
    grid_A_rho_2[A] += 2 * np.einsum("tgk , rgl, kl -> trg", grid_ao_1[:, :, sA],    grid_ao_1, D[sA], optimize=True)

# NoU GGA
hess_noU_gga_1  = 1 * np.einsum("g, Atg , Bsg          -> ABts", grid_frr, grid_A_rho_1, grid_A_rho_1,                         optimize=True)
hess_noU_gga_1 += 2 * np.einsum("g, Atg , Bswg, wg     -> ABts", grid_frg, grid_A_rho_1, grid_A_rho_2, grid_rho_1,             optimize=True)
hess_noU_gga_1 += 2 * np.einsum("g, Atrg, Bsg , rg     -> ABts", grid_frg, grid_A_rho_2, grid_A_rho_1, grid_rho_1,             optimize=True)
hess_noU_gga_1 += 4 * np.einsum("g, Atrg, Bswg, rg, wg -> ABts", grid_fgg, grid_A_rho_2, grid_A_rho_2, grid_rho_1, grid_rho_1, optimize=True)
hess_noU_gga_1 += 2 * np.einsum("g, Atrg, Bsrg         -> ABts", grid_fg,  grid_A_rho_2, grid_A_rho_2,                         optimize=True)

hess_noU_gga_2 = np.zeros((natm, natm, 3, 3))
hess_noU_gga_2_ao  = 4 * np.einsum("g, rg, trgu, sgv -> tsuv", grid_fg, grid_rho_1, grid_ao_2, grid_ao_1, optimize=True)
hess_noU_gga_2_ao += hess_noU_gga_2_ao.transpose(1, 0, 3, 2)
hess_noU_gga_2_ao += 2 * np.einsum("g, tgu, sgv -> tsuv", grid_fr, grid_ao_1, grid_ao_1, optimize=True)

for A in range(natm):
    for B in range(A, natm):
        sA, sB = mol_slice(A), mol_slice(B)
        hess_noU_gga_2[A, B] += np.einsum("tsuv, uv -> ts", hess_noU_gga_2_ao[:, :, sA, sB], D[sA, sB])
        hess_noU_gga_2[B, A] = hess_noU_gga_2[A, B].T

hess_noU_gga_3_ao  = 2 * np.einsum("g,      Tgu,  gv -> Tuv", grid_fr, grid_ao_2T, grid_ao_0, optimize=True)
hess_noU_gga_3_ao += 4 * np.einsum("g, rg,  Tgu, rgv -> Tuv", grid_fg, grid_rho_1, grid_ao_2T, grid_ao_1, optimize=True)
hess_noU_gga_3_ao += 4 * np.einsum("g, rg, rTgu,  gv -> Tuv", grid_fg, grid_rho_1, grid_ao_3T, grid_ao_0, optimize=True)

XX, XY, XZ, YY, YZ, ZZ = range(6)
hess_noU_gga_3 = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    sA = mol_slice(A)
    hess_noU_gga_3[A, A] = np.einsum("Tuv, uv -> T", hess_noU_gga_3_ao[:, sA], D[sA])[[XX, XY, XZ, XY, YY, YZ, XZ, YZ, ZZ]].reshape((3, 3))
    
# U GGA

hess_ao_hgga  = -0.5 * np.einsum("g,     Atg ,      gu, gv -> Atuv", grid_frr,               grid_A_rho_1,             grid_ao_0, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -  2 * np.einsum("g,     Atg , wg, wgu, gv -> Atuv", grid_frg,               grid_A_rho_1, grid_rho_1, grid_ao_1, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -      np.einsum("g, rg, Atrg,      gu, gv -> Atuv", grid_frg, grid_rho_1,   grid_A_rho_2,             grid_ao_0, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -  4 * np.einsum("g, rg, Atrg, wg, wgu, gv -> Atuv", grid_fgg, grid_rho_1,   grid_A_rho_2, grid_rho_1, grid_ao_1, grid_ao_0, optimize=EINOPT)
hess_ao_hgga += -  2 * np.einsum("g,     Atrg,     rgu, gv -> Atuv", grid_fg,                grid_A_rho_2,             grid_ao_1, grid_ao_0, optimize=EINOPT)
hess_ao_hgga_mat1  = -   np.einsum("g,      tgu,  gv -> tuv", grid_fr,             grid_ao_1, grid_ao_0, optimize=EINOPT)
hess_ao_hgga_mat1 -= 2 * np.einsum("g, rg,  tgu, rgv -> tuv", grid_fg, grid_rho_1, grid_ao_1, grid_ao_1, optimize=EINOPT)
hess_ao_hgga_mat1 -= 2 * np.einsum("g, rg, trgu,  gv -> tuv", grid_fg, grid_rho_1, grid_ao_2, grid_ao_0, optimize=EINOPT)
for A in range(natm):
    sA = mol_slice(A)
    hess_ao_hgga[A, :, sA] += hess_ao_hgga_mat1[:, sA]
hess_ao_hgga += hess_ao_hgga.swapaxes(2, 3)

grid_rho1phi1 = np.einsum("tg, tgu -> gu", grid_rho_1, grid_ao_1, optimize=True)
rdm2_inAx  =   2 * np.einsum("g,  gk,  gu -> gku", grid_frg, grid_ao_0,     grid_rho1phi1, optimize=EINOPT)
rdm2_inAx += rdm2_inAx.swapaxes(1, 2)
rdm2_inAx += 0.5 * np.einsum("g,  gk,  gu -> gku", grid_frr, grid_ao_0,     grid_ao_0,     optimize=EINOPT)
rdm2_inAx +=   8 * np.einsum("g,  gk,  gu -> gku", grid_fgg, grid_rho1phi1, grid_rho1phi1, optimize=EINOPT)
rdm2_inAx +=   4 * np.einsum("g, rgk, rgu -> gku", grid_fg,  grid_ao_1,     grid_ao_1,     optimize=EINOPT)
rdm2_inAx = np.einsum("gku, gl, gv -> kluv", rdm2_inAx, grid_ao_0, grid_ao_0, optimize=EINOPT)

def Ax(x):
    shape = x.shape
    x = x.reshape((-1, nmo, nocc))
    dx = C @ x @ Co.T
    v = np.zeros_like(x)
    for i in range(dx.shape[0]):
        dm = dx[i] + dx[i].T
        v[i] = C.T @ (scf_eng.get_j(mol, dm) - 0.5 * c_x * scf_eng.get_k(mol, dm)) @ Co
        vks = np.einsum("kluv, kl -> uv", rdm2_inAx, dm)
        v[i] += C.T @ (vks + vks.T) @ Co
    return 2 * v.reshape(shape)

hess_ao_h1 = np.array([ get_hess_ao_h1(A) + vxc_deriv1_pyscf[A] for A in range(natm) ])
hess_ao_s1 = np.array([ get_hess_ao_s1(A) for A in range(natm) ])
hess_pi_h1 = np.einsum("Atuv, up, vi -> Atpi", hess_ao_h1, C, Co)
hess_pi_s1 = np.einsum("Atuv, up, vi -> Atpi", hess_ao_s1, C, Co)

hess_U, hess_M = scf.cphf.solve(Ax, e, mo_occ, hess_pi_h1.reshape(-1, nmo, nocc), hess_pi_s1.reshape(-1, nmo, nocc))
hess_U.shape = (natm, 3, nmo, nocc); hess_M.shape = (natm, 3, nocc, nocc)

hess_withU = 4 * np.einsum("Bspi, Atpi -> ABts", hess_U, hess_pi_h1)
hess_withU -= 4 * np.einsum("Bspi, Atpi, i -> ABts", hess_U, hess_pi_s1, eo)
hess_withU -= 2 * np.einsum("Atki, Bski -> ABts", hess_pi_s1[:, :, :nocc], hess_M)

hess_my_GGA_elec = hess_withU + hess_noU_noGGA + hess_noU_gga_1 + hess_noU_gga_2 + hess_noU_gga_3

In [ ]:
%%time
hess_pyscf_GGA_elec = scf_hess.hess_elec()

In [ ]:
np.allclose(hess_my_GGA_elec, hess_pyscf_GGA_elec)